# Power Assignment

In [ ]:
# Imports
import numpy as np
import cvxpy as cp

## Given

In [ ]:
G = np.array([
    [1.0, 0.1, 0.2, 0.1, 0.0],
    [0.1, 1.0, 0.1, 0.1, 0.0],
    [0.2, 0.1, 2.0, 0.2, 0.2],
    [0.1, 0.1, 0.2, 1.0, 0.1],
    [0.0, 0.0, 0.2, 0.1, 1.0]
])
n, _ = G.shape

p_min = 0
p_max = 3

groups = [[1, 2], [3, 4, 5]]
p_rc = [4, 6]

sigma = 0.5
p_received_max = 5

# For alpha
epsilon = 0.05
alpha_max = 1e3 # An arbitary max
alpha_min = 0


## Framing the Problem


In [ ]:
# Variables and Parameters
p = cp.Variable(n)
S = cp.multiply(p, G.diagonal())
I = G@p - S  # Axis = 1 is sum of rows
alpha = cp.Parameter(shape=1)

# Objective
objective = cp.Minimize(alpha)

# Constraints
constraints = [
    # Alpha Constraint
    I + sigma <= alpha * S,

    # Power Constraints
    p >= p_min,
    p <= p_max,

    # Received Max Power
    G @ p <= p_received_max
]

# Constraints for groups
for i in range(len(groups)):
    group = groups[i]
    temp = np.zeros(n)
    for ele in group:
        temp[ele - 1] = 1
    constraints += [cp.sum(cp.multiply(temp, p)) <= p_rc[i]]

# Problem
prob = cp.Problem(objective, constraints)



## Solving

In [ ]:
# Solving using Bisection Menthod

def optimize_solution(a_max, a_min):
    alpha.value = [a_max]
    prob.solve()
    if prob.status != 'optimal':
        return
    alpha.value = [a_min]
    prob.solve()
    if prob.status == 'optimal':
        return

    opt_powers = None
    while a_max - a_min > epsilon:
        alpha.value = np.atleast_1d((a_max + a_min) / 2.0)
        prob.solve()
        if prob.status == 'optimal':
            a_max = alpha.value
            opt_powers = p.value
        else:
            a_min = alpha.value

    return opt_powers


optimal_powers = optimize_solution(alpha_max, alpha_min)
print("Optimal Solution:", 1 / float(alpha.value))
print("Optimal Powers:", optimal_powers)


Optimal Solution: 1.7246315789473685
Optimal Powers: [2.05325577 1.85690341 1.5972308  2.3231306  1.85569467]
